# Parse gene-disease associations from OrphaNet

2017-06-08

1. Parse OrphaNet XML files for gene-disease links and gene cross-references.
2. Map genes to HGNC identifiers.
3. Map genes from HGNC ids to Entrez ids.

In [1]:
import xml.etree.ElementTree as ET
from collections import defaultdict

import pandas as pd
import numpy as np

## Create OrphaNet XML parsers

In [2]:
def parse_links(root):
    """Parse out gene-disease links."""
    links = defaultdict(list)

    for disease in root.iter("Disorder"):
        dise_id = disease.find("OrphaNumber").text
        dise_name = disease.find("Name").text

        for link in disease.iterfind("DisorderGeneAssociationList/DisorderGeneAssociation"):
            link_type = link.find("DisorderGeneAssociationType/Name").text
            link_status = link.find("DisorderGeneAssociationStatus/Name").text

            gene_id = link.find("Gene/OrphaNumber").text

            # create gene-dise link
            vals = ["dise_id", "dise_name", "gene_id", "link_type", "link_status"]
            for vname in vals:
                links[vname].append(locals()[vname])
        
    return (pd
        .DataFrame(links)
        .astype({"dise_id": np.int64, "gene_id": np.int64})
    )

In [3]:
def parse_xrefs(root):
    """Parse out gene identifier cross references."""
    
    res = defaultdict(list)
    
    for link in root.iter("DisorderGeneAssociation"):
        gene = link.find("Gene")
        
        gene_id = gene.find("OrphaNumber").text
        gene_name = gene.find("Name").text
        gene_symb = gene.find("Symbol").text
        
        for xref in gene.iterfind("ExternalReferenceList/ExternalReference"):
            ref_name = xref.find("Source").text
            ref_id = xref.find("Reference").text
            
            vals = ["gene_id", "gene_name", "gene_symb", "ref_name", "ref_id"]
            for vname in vals:
                res[vname].append(locals()[vname])
        
    return (pd
        .DataFrame(res)
        .drop_duplicates()
        .astype({"gene_id": np.int64})
    )

---

## Extract information

In [4]:
tree = ET.parse("../data/raw/orphanet/gene_disease_links.xml")

root = tree.getroot()

In [5]:
links = parse_links(root)
xrefs = parse_xrefs(root)

In [6]:
links.shape

(6977, 5)

In [7]:
xrefs.shape

(21702, 5)

In [8]:
links.head()

,dise_id,dise_name,gene_id,link_status,link_type
0,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",268061,Assessed,Disease-causing germline mutation(s) in
1,93,Aspartylglucosaminuria,119513,Assessed,Disease-causing germline mutation(s) in
2,585,Multiple sulfatase deficiency,119899,Assessed,Disease-causing germline mutation(s) in
3,118,Beta-mannosidosis,123131,Assessed,Disease-causing germline mutation(s) in
4,166068,Pontocerebellar hypoplasia type 5,168268,Assessed,Disease-causing germline mutation(s) in


In [9]:
xrefs.head()

,gene_id,gene_name,gene_symb,ref_id,ref_name
0,268061,kinesin family member 7,KIF7,30497,HGNC
1,268061,kinesin family member 7,KIF7,611254,OMIM
2,268061,kinesin family member 7,KIF7,KIF7,Genatlas
3,268061,kinesin family member 7,KIF7,Q2M1P5,SwissProt
4,268061,kinesin family member 7,KIF7,ENSG00000166813,Ensembl


In [10]:
xrefs.groupby(["gene_id", "ref_name"])["ref_id"].nunique().value_counts()

1    21663
2       11
5        2
4        1
3        1
Name: ref_id, dtype: int64

One gene may have multiple protein products, which leads to the multiple identifiers for the cross references. I have manually verified that these are ok.

---

## Check for data consistency

In [11]:
links["gene_id"].nunique()

3713

In [12]:
xrefs["gene_id"].nunique()

3712

In [13]:
set(links["gene_id"]) - set(xrefs["gene_id"])

{398677}

One gene in OrphaNet has no Xrefs to other databases and can therefore be discarded.

In [14]:
xrefs.groupby("ref_name")["gene_id"].nunique().sort_values(ascending=False)

ref_name
HGNC         3711
Ensembl      3674
OMIM         3667
SwissProt    3646
Genatlas     3641
Reactome     2867
IUPHAR        472
Name: gene_id, dtype: int64

HGNC identifiers exist for all but one of the genes. The missing gene can be discarded.

In [15]:
xrefs.groupby("gene_id").filter(lambda df: 'HGNC' not in df["ref_name"].values)

,gene_id,gene_name,gene_symb,ref_id,ref_name
30593,455932,"DNase1 hypersensitivity, chromosome 6, site 1",DHS6S1,616842,OMIM


Conclusion: two genes in the gene-disease links do not have HGNC ids, and will be discarded.

---

## Link information

In [16]:
links["dise_id"].nunique()

3605

In [17]:
links["link_status"].value_counts()

Assessed            6514
Not yet assessed     463
Name: link_status, dtype: int64

In [18]:
links["link_type"].value_counts()

Disease-causing germline mutation(s) in                       4318
Disease-causing germline mutation(s) (loss of function) in     993
Major susceptibility factor in                                 478
Candidate gene tested in                                       256
Part of a fusion gene in                                       222
Role in the phenotype of                                       221
Disease-causing germline mutation(s) (gain of function) in     198
Disease-causing somatic mutation(s) in                         161
Biomarker tested in                                             92
Modifying germline mutation in                                  38
Name: link_type, dtype: int64

---

## Map gene-disease links to Entrez space

### Convert Orphanet gene ids to HGNC ids

In [19]:
good = (xrefs
    .query("ref_name == 'HGNC'")
    .drop("ref_name", axis=1)
    .rename(columns={"ref_id": "hgnc_id"})
    .astype({"hgnc_id": np.int64})
)

In [20]:
good.head()

,gene_id,gene_name,gene_symb,hgnc_id
0,268061,kinesin family member 7,KIF7,30497
7,119513,aspartylglucosaminidase,AGA,318
13,119899,sulfatase modifying factor 1,SUMF1,20376
19,123131,mannosidase beta,MANBA,6831
25,168268,tRNA splicing endonuclease subunit 54,TSEN54,27561


In [21]:
id_map = pd.read_csv("hgnc_entrez_map.tsv", sep='\t')

In [22]:
id_map.head()

,hgnc_id,symbol,status,entrez_id
0,5,A1BG,Approved,1.0
1,37133,A1BG-AS1,Approved,503538.0
2,24086,A1CF,Approved,29974.0
3,7,A2M,Approved,2.0
4,27057,A2M-AS1,Approved,144571.0


### Check that all genes have an Entrez id

In [23]:
set(good["hgnc_id"]) <= set(
    id_map.query("status == 'Approved'").dropna(how="any")["hgnc_id"]
)

True

All genes involved in gene-disease links have Entrez gene ids.

### Map HGNC ids to Entrez Gene

In [24]:
# 22 gene symbols do not match; we will use the HGNC symbol to override disagreements

gene_map = (good
    .merge(
        id_map.dropna(how="any").astype({"entrez_id": np.int64}),
        how="left", on="hgnc_id"
    )
    .drop(["status", "gene_symb"], axis=1)
)            

In [25]:
gene_map.head()

,gene_id,gene_name,hgnc_id,symbol,entrez_id
0,268061,kinesin family member 7,30497,KIF7,374654
1,119513,aspartylglucosaminidase,318,AGA,175
2,119899,sulfatase modifying factor 1,20376,SUMF1,285362
3,123131,mannosidase beta,6831,MANBA,4126
4,168268,tRNA splicing endonuclease subunit 54,27561,TSEN54,283989


## Convert gene ids in gene-disease link info to Entrez ids

In [26]:
fin_links = (links
    .merge(gene_map, how="inner", on="gene_id")
    .drop("gene_id", axis=1)
    .rename(columns={"dise_id": "dise_orpha_id"})
)

In [27]:
fin_links.head()

,dise_orpha_id,dise_name,link_status,link_type,gene_name,hgnc_id,symbol,entrez_id
0,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",Assessed,Disease-causing germline mutation(s) in,kinesin family member 7,30497,KIF7,374654
1,36,Acrocallosal syndrome,Assessed,Disease-causing germline mutation(s) in,kinesin family member 7,30497,KIF7,374654
2,2189,Hydrolethalus,Assessed,Disease-causing germline mutation(s) in,kinesin family member 7,30497,KIF7,374654
3,2754,Joubert syndrome with orofaciodigital defect,Assessed,Disease-causing germline mutation(s) in,kinesin family member 7,30497,KIF7,374654
4,93,Aspartylglucosaminuria,Assessed,Disease-causing germline mutation(s) in,aspartylglucosaminidase,318,AGA,175


In [28]:
fin_links.shape

(6975, 8)

In [29]:
fin_links["dise_orpha_id"].nunique()

3603

In [30]:
fin_links["entrez_id"].nunique()

3711

In [31]:
fin_links[["dise_orpha_id", "entrez_id"]].drop_duplicates().shape

(6947, 2)

Different mutations in the same gene may cause the disease, leading to more rows than there are gene-disease pairs. This is ok, and can be constructed as different edges in the final graph.

## Save to file

In [32]:
fin_links.to_csv("orpha_gene_dise_links.tsv", sep='\t', index=False)